<a href="https://colab.research.google.com/github/Karthick47v2/speech-emotion-classifier/blob/main/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## Import Libs


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, f1_score
from tensorflow import keras
from tensorflow.keras.utils import to_categorical


## Load and split data


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [3]:
X_mfcc = np.load('drive/MyDrive/SER/X_mfcc_2048.npy')
X_mel = np.load('drive/MyDrive/SER/X_mel_2048.npy')
y = np.load('drive/MyDrive/SER/y_2048.npy')


In [4]:
le = LabelEncoder()
y_encoded = to_categorical(le.fit_transform(y))


In [17]:
x_train, x_test, y_train, y_test = train_test_split(
    X_mel, y_encoded, random_state=42, test_size=0.1)
x_test, x_T_test, y_test, y_T_test = train_test_split(
    x_test, y_test, random_state=42, test_size=0.5)


In [ ]:
n_rows = x_train.shape[1]
n_cols = x_train.shape[2]
n_channels = 1

x_train = x_train.reshape(x_train.shape[0], n_rows, n_cols, n_channels)
x_test = x_test.reshape(x_test.shape[0], n_rows, n_cols, n_channels)


## Train


### Model - Log-MEL


In [ ]:
# # MEL - 300 EPOCHS
model = keras.Sequential()

model.add(keras.layers.Conv2D(filters=16, kernel_size=(5, 5), kernel_initializer="he_normal",
          input_shape=(n_rows, n_cols, n_channels), activation='relu'))
model.add(keras.layers.BatchNormalization())

model.add(keras.layers.MaxPooling2D(pool_size=(6, 6)))

model.add(keras.layers.Conv2D(filters=32, kernel_size=(3, 3),
          activation='relu', kernel_initializer="he_normal"))
model.add(keras.layers.BatchNormalization())

model.add(keras.layers.MaxPooling2D(pool_size=(6, 6)))

model.add(keras.layers.Conv2D(filters=48, kernel_size=(3, 3),
          activation='relu', kernel_initializer="he_normal"))
model.add(keras.layers.BatchNormalization())


model.add(keras.layers.Dropout(0.7))

model.add(keras.layers.GlobalAveragePooling2D())
model.add(keras.layers.Dense(units=7, activation='softmax'))

model.compile(loss='categorical_crossentropy', metrics=[
              'accuracy'], optimizer=keras.optimizers.Adam(learning_rate=2e-4))  # 2

# model.summary()
n_epochs = 300
batch_size = 32

checkpoint = keras.callbacks.ModelCheckpoint(filepath='mel-model',
                                             monitor='val_loss',
                                             mode='min',
                                             save_best_only=True)

history = model.fit(x_train, y_train, batch_size=batch_size, epochs=n_epochs,
                    validation_data=(x_test, y_test), callbacks=[checkpoint], verbose=1)


Plot metrics


In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, label='Training acc')
plt.plot(epochs, val_acc, label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, label='Training loss')
plt.plot(epochs, val_loss, label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()


### Model - MFCC


In [ ]:
# MFCC - 300 epochs
model = keras.Sequential()

model.add(keras.layers.Conv2D(filters=16, kernel_size=(3, 3), kernel_initializer="he_normal",
          input_shape=(n_rows, n_cols, n_channels), activation='relu'))
model.add(keras.layers.BatchNormalization())

model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))

model.add(keras.layers.Conv2D(filters=32, kernel_size=(3, 3),
          activation='relu', kernel_initializer="he_normal"))
model.add(keras.layers.BatchNormalization())

model.add(keras.layers.Conv2D(filters=48, kernel_size=(3, 3),
          activation='relu', kernel_initializer="he_normal"))
model.add(keras.layers.BatchNormalization())

model.add(keras.layers.Dropout(0.7))

model.add(keras.layers.GlobalAveragePooling2D())
model.add(keras.layers.Dense(units=7, activation='softmax'))

model.compile(loss='categorical_crossentropy', metrics=[
              'accuracy'], optimizer=keras.optimizers.Adam(learning_rate=4e-4))  # 2

# model.summary()
n_epochs = 300
batch_size = 32

checkpoint = keras.callbacks.ModelCheckpoint(filepath='mfcc-model',
                                             monitor='val_loss',
                                             mode='min',
                                             save_best_only=True)

history = model.fit(x_train, y_train, batch_size=batch_size, epochs=n_epochs,
                    validation_data=(x_test, y_test), callbacks=[checkpoint], verbose=1)


Plot metrics


In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, label='Training acc')
plt.plot(epochs, val_acc, label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, label='Training loss')
plt.plot(epochs, val_loss, label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()


## Test


In [ ]:
! unzip mel_2048.zip
! unzip mfcc_2048.zip


Log-MEL


In [13]:
# test
mel_model = keras.models.load_model('mel_2048')

x_train, x_test, y_train, y_test = train_test_split(
    X_mel, y_encoded, random_state=42, test_size=0.1)
x_test, x_MEL_test, y_test, y_MEL_test = train_test_split(
    x_test, y_test, random_state=42, test_size=0.5)


In [ ]:
y_preds = mel_model.predict(x_MEL_test)
y_hat = np.argmax(y_preds, axis=1)
y_true = np.argmax(y_MEL_test, axis=1)

cm = confusion_matrix(y_true, y_hat)

sum = 0

for i in range(cm.shape[0]):
    sum += cm[i, i]

print(len(y_preds))
print(sum / len(y_preds))
print(f1_score(y_true, y_hat, average='weighted'))
sns.heatmap(cm, annot=True)


MFCC


In [15]:
mfcc_model = keras.models.load_model('mfcc_2048')

x_train, x_test, y_train, y_test = train_test_split(
    X_mfcc, y_encoded, random_state=42, test_size=0.1)
x_test, x_MFCC_test, y_test, y_MFCC_test = train_test_split(
    x_test, y_test, random_state=42, test_size=0.5)


In [ ]:
y_preds = mfcc_model.predict(x_MFCC_test)
y_hat = np.argmax(y_preds, axis=1)
y_true = np.argmax(y_MFCC_test, axis=1)

cm = confusion_matrix(y_true, y_hat)

sum = 0

for i in range(cm.shape[0]):
    sum += cm[i, i]

print(len(y_preds))
print(sum / len(y_preds))
print(f1_score(y_true, y_hat, average='weighted'))
sns.heatmap(cm, annot=True)


Ensembled model (Bagging)


In [ ]:
y1_preds = mfcc_model.predict(x_MFCC_test)
y2_preds = mel_model.predict(x_MEL_test)
y_preds = y1_preds * 0.7 + y2_preds * 0.3
y_hat = np.argmax(y_preds, axis=1)
y_true = np.argmax(y_T_test, axis=1)

cm = confusion_matrix(y_true, y_hat)

sum = 0

for i in range(cm.shape[0]):
    sum += cm[i, i]

print(len(y_preds))
print(sum / len(y_preds))
print(f1_score(y_true, y_hat, average='weighted'))
sns.heatmap(cm, annot=True)
